## 2 Ways to Use Deployed CML Model in Development

Look at https://docs.cloudera.com/machine-learning/cloud/api/topics/ml-apiv2-usage-examples.html for documentation on Python library.

### 1. Use CML Python Library to Load Model Inputs

#### Launch CML client utility

In [37]:
import cmlapi
import json
import sys
import os
client = cmlapi.default_client(url=os.getenv("CDSW_API_URL").replace("/api/v1", ""), cml_api_key=os.getenv("CDSW_APIV2_KEY"))
client.list_projects()

{'next_page_token': '',
 'projects': [{'created_at': datetime.datetime(2023, 12, 21, 0, 56, 5, 481985, tzinfo=tzlocal()),
               'creation_status': 'success',
               'creator': {'email': 'ktalbert@cloudera.com',
                           'name': 'Kevin Talbert',
                           'username': 'ktalbert'},
               'default_engine_type': 'ml_runtime',
               'description': 'This AMP deploys LLAMA 2 7 billion token model '
                              'as a CML model, with replicas. Requires GPUs.\n',
               'environment': '{"MODEL_NAME":"LLAMA-2-7B '
                              'CML","CDSW_APP_POLLING_ENDPOINT":"/","PROJECT_OWNER":"ktalbert"}',
               'ephemeral_storage_limit': 20,
               'ephemeral_storage_request': 0,
               'id': '0i8q-r17t-8r3a-o5db',
               'name': 'Shared LLM Model for Hands on Lab - ktalbert',
               'owner': {'email': 'ktalbert@cloudera.com',
                         'name'

#### Locate and assign your CML project

In [38]:
projects = client.list_projects(search_filter=json.dumps({"name": "Shared LLM Model for Hands on Lab"}))
project = projects.projects[0] # assuming only one project is returned by the above query
print(project)

{'created_at': datetime.datetime(2023, 12, 21, 0, 56, 5, 481985, tzinfo=tzlocal()),
 'creation_status': 'success',
 'creator': {'email': 'ktalbert@cloudera.com',
             'name': 'Kevin Talbert',
             'username': 'ktalbert'},
 'default_engine_type': 'ml_runtime',
 'description': 'This AMP deploys LLAMA 2 7 billion token model as a CML '
                'model, with replicas. Requires GPUs.\n',
 'environment': '{"MODEL_NAME":"LLAMA-2-7B '
                'CML","CDSW_APP_POLLING_ENDPOINT":"/","PROJECT_OWNER":"ktalbert"}',
 'ephemeral_storage_limit': 20,
 'ephemeral_storage_request': 0,
 'id': '0i8q-r17t-8r3a-o5db',
 'name': 'Shared LLM Model for Hands on Lab - ktalbert',
 'owner': {'email': 'ktalbert@cloudera.com',
           'name': 'Kevin Talbert',
           'username': 'ktalbert'},
 'permissions': {'admin': True,
                 'business_user': True,
                 'inherit': False,
                 'operator': True,
                 'read': True,
                 'wr

####  Locate CML model and Load Access Key for Model to Environment

In [39]:
## Here we assume that only one model has been deployed in the project, if this is not true this should be adjusted (this is reflected by the placeholder 0 in the array)
model = client.list_models(project_id=project.id)
selected_model = model.models[0]

## Save the access key for the model to the environment variable of this project
os.environ["MODEL_ACCESS_KEY"] = selected_model.access_key

#### Generate Model Endpoint URL for Request

In [40]:
MODEL_ENDPOINT = os.getenv("CDSW_API_URL").replace("https://", "https://modelservice.").replace("/api/v1", "/model?accessKey=")
MODEL_ENDPOINT = MODEL_ENDPOINT + os.environ["MODEL_ACCESS_KEY"]
os.environ["MODEL_ENDPOINT"] = MODEL_ENDPOINT
print(MODEL_ENDPOINT)

https://modelservice.ml-c2033a45-771.se-sandb.a465-9q4k.cloudera.site/model?accessKey=mrxixviq0atro0z1fdreqvw7peq6dhc7


### 2. Create Request to Model

In [43]:
## Write a question to ask the model
question = "What is Cloudera Data Platform?"

In [44]:
import requests
import json
import os

## Set variables
temperature = 0.01
token_count = 150

llama_sys = f"<<SYS>>\n You are a helpful and honest assistant. If you are unsure about an answer, truthfully say \"I don't know\".\n<</SYS>>\n\n"
llama_inst = f"[INST]Use your knowledge to answer the user's question. [/INST]"
question_and_context = f"{llama_sys} {llama_inst} [INST] User: {question} [/INST]"

data={ "request": {"prompt":question_and_context,"temperature":temperature,"max_new_tokens":token_count,"repetition_penalty":1.0} }

r = requests.post(os.environ["MODEL_ENDPOINT"], data=json.dumps(data), headers={'Content-Type': 'application/json'})

# Logging
print(f"Request: {data}")
print(f"Response: {r.json()}")


Request: {'request': {'prompt': '<<SYS>>\n You are a helpful and honest assistant. If you are unsure about an answer, truthfully say "I don\'t know".\n<</SYS>>\n\n [INST]Use your knowledge to answer the user\'s question. [/INST] [INST] User: What is Cloudera Data Platform? [/INST]', 'temperature': 0.01, 'max_new_tokens': 150, 'repetition_penalty': 1.0}}
Response: {'success': True, 'response': {'model_deployment_crn': 'crn:cdp:ml:us-west-1:558bc1d2-8867-4357-8524-311d51259233:workspace:2a01dba0-7ce6-4a14-97b3-551b694314f8/b91f63eb-502f-4895-9c06-24546439e37c', 'prediction': {'response': '<<SYS>>\n You are a helpful and honest assistant. If you are unsure about an answer, truthfully say "I don\'t know".\n<</SYS>>\n\n [INST]Use your knowledge to answer the user\'s question. [/INST] [INST] User: What is Cloudera Data Platform? [/INST]  Cloudera Data Platform is a comprehensive, distributed data management and analytics platform that enables organizations to collect, store, process, and ana